In [ ]:
!pip install -qU \
    qdrant-client \
    sentence-transformers \
    transformers \
    beautifulsoup4 \
    requests \
    tqdm \
    langchain \
    accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.7/327.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import json
from bs4 import BeautifulSoup
import requests
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import PointStruct
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import List, Dict, Tuple
import time

In [ ]:
def segment_text(text: str, max_length: int = 512) -> List[str]:

    paragraphs = text.split('\n')
    segments = []
    current_segment = []
    current_length = 0

    for paragraph in paragraphs:
        paragraph = paragraph.strip()
        if not paragraph:
            continue

        if current_length + len(paragraph) + 1 <= max_length:
            current_segment.append(paragraph)
            current_length += len(paragraph) + 1
        else:
            if current_segment:
                segments.append('\n'.join(current_segment))
                current_segment = [paragraph]
                current_length = len(paragraph)
            else:
                segments.append(paragraph)
                current_length = 0

    if current_segment:
        segments.append('\n'.join(current_segment))

    return segments

In [ ]:
# Подготовка сегментов для индексации
def prepare_segments(data: Dict[str, str]) -> List[Dict]:
    segments = []
    segment_id = 0

    for article_name, content in data.items():
        article_segments = segment_text(content)

        for i, segment in enumerate(article_segments):
            segments.append({
                "id": segment_id,
                "article": article_name,
                "segment_num": i,
                "text": segment,
                "metadata": {
                    "article": article_name,
                    "segment": i,
                    "total_segments": len(article_segments)
                }
            })
            segment_id += 1

    return segments

In [ ]:
# Создание базы данных Qdrant
class VectorDatabase:
    def __init__(self, collection_name: str = "ml_handbook", model_name: str = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"):
        self.client = QdrantClient(":memory:")
        self.model = SentenceTransformer(model_name)
        self.collection_name = collection_name
        self.dim = self.model.get_sentence_embedding_dimension()

    def create_collection(self):
        self.client.recreate_collection(
            collection_name=self.collection_name,
            vectors_config=models.VectorParams(
                size=self.dim,
                distance=models.Distance.COSINE
            )
        )

    def index_segments(self, segments: List[Dict]):
        points = []

        for segment in segments:
            embedding = self.model.encode(segment["text"]).tolist()

            points.append(PointStruct(
                id=segment["id"],
                vector=embedding,
                payload={
                    "text": segment["text"],
                    "article": segment["article"],
                    "segment_num": segment["segment_num"],
                    "metadata": segment["metadata"]
                }
            ))

        self.client.upsert(
            collection_name=self.collection_name,
            points=points
        )

    def search(self, query: str, top_k: int = 3) -> List[Dict]:
        query_embedding = self.model.encode(query).tolist()

        results = self.client.search(
            collection_name=self.collection_name,
            query_vector=query_embedding,
            limit=top_k
        )

        return [{
            "score": hit.score,
            "text": hit.payload["text"],
            "article": hit.payload["article"],
            "segment_num": hit.payload["segment_num"]
        } for hit in results]

In [ ]:
# Модель для генерации ответов
class TinyLlama:
    def __init__(self, model_name: str = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
            device_map="auto"
        )

    def generate_answer(self, question: str, context: str = None, max_length: int = 512) -> str:
        messages = []

        if context:
            messages.append({
                "role": "system",
                "content": f"Ты - помощник по машинному обучению. Отвечай на вопросы, используя предоставленный контекст:\n{context}"
            })

        messages.append({
            "role": "user",
            "content": question
        })

        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        model_inputs = self.tokenizer([text], return_tensors="pt").to(self.device)

        generated_ids = self.model.generate(
            model_inputs.input_ids,
            max_new_tokens=max_length,
            pad_token_id=self.tokenizer.eos_token_id
        )

        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(
                model_inputs.input_ids,
                generated_ids
            )
        ]

        response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        return response.strip()

In [ ]:
# RAG система
class RAGSystem:
    def __init__(self):
        self.vector_db = VectorDatabase()
        self.llm = TinyLlama()
        self.vector_db.create_collection()

    def initialize(self, data_path: str):
        with open(data_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        segments = prepare_segments(data)
        self.vector_db.index_segments(segments)

    # Поиск релевантных фрагментов
    def answer_with_rag(self, question: str) -> Tuple[str, List[Dict]]:
        search_results = self.vector_db.search(question)

        context = "\n\n".join([res["text"] for res in search_results])
        answer = self.llm.generate_answer(question, context)

        return answer, search_results

    # Ответ без контекста
    def answer_without_rag(self, question: str) -> str:
        return self.llm.generate_answer(question)

In [ ]:
rag = RAGSystem()
rag.initialize("handbook_articles.json")

<ipython-input-31-e65552691808>:10: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  self.client.recreate_collection(


## Вопрос №1: Что показывает ROC-кривая?


In [ ]:
answer = rag.answer_with_rag('Что показывает ROC-кривая?')

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(


In [ ]:
answer[1]

[{'score': 0.7177155737883694,
  'text': 'Поэкспериментируйте с разными вариантами распределения предсказаний по классам и посмотрите, как меняется ROC-кривая.\nЧем лучше классификатор разделяет два класса, тем больше площадь (area under curve) под ROC-кривой — и мы можем использовать её в качестве метрики. Эта метрика называетсяAUCи она работает благодаря следующему свойству ROC-кривой:\nAUCравен доле пар объектов вида (объект класса 1, объект класса 0), которые алгоритм верно упорядочил, то есть предсказание классификатора на первом объекте больше:',
  'article': 'Метрики классификации и регрессии',
  'segment_num': 53},
 {'score': 0.6946442526940309,
  'text': 'Если классификатор идеальный, — две кривые разделимы по оси X, — то на правом графике мы получаем ROC-кривую (0,0)->(0,1)->(1,1), площадь под которой равна 1.\nЕсли классификатор случайный (предсказывает одинаковые метки положительным и отрицательным объектам), то мы получаем ROC-кривую (0,0)->(1,1), площадь под которой равна

In [ ]:
answer[0]

'ROC-кривая (Receiver Operating Characteristic Curve, или кривая получения) представляет собой график, который показывает отношение плотности (распределения) предсказаний к плотности (распределения) объектов, которые были классифицированы правильно. ROC-кривая имеет две оси, X и Y, и имеет две линии, которые проходят через точки X и Y.\n\nПервая линия (X) проходит через точку Y, показывающую плотность объектов, которые были классифицированы правильно. Вторая линия (Y) проходит через точку X, показывающую плотность объектов, которые были классифицированы неправильно.\n\nОбъекты, которые были классифицированы правильно, имеют значение 1, а объекты, которые были классифицированы неправильно, имеют значение 0. ROC-кривая показывает, как плотность объектов, которые были классифицированы правильно, влияет на плотность объектов, которые были классифицированы неправильно.\n\nПри варьировании порога ROC-кривая меняется, показывая, как плотность объектов, которые были классифицированы правильно,

In [ ]:
rag.answer_without_rag('Что показывает ROC-кривая?')

'ROC-кривая (Receiver Operating Characteristic Curve) описывает способность классификатора (например, классификатора решения задачи) набора признаков (например, признаков классификации) на определенных выборках (например, наборов данных) для определения оптимального значения параметра, который будет использоваться для классификации данных. ROC-кривая показывает, на каком протяжении распределения значений признаков распределение значений классифицируемых признаков соответствует распределению значений классифицируемых признаков на границе между классами. Если ROC-кривая имеет линейный ответ, то классификатор будет способен классифицировать данные на основе линейной функции распределения признаков. Если ROC-кривая имеет нелинейный ответ, то классификатор будет способен классифицировать данные на основе нелинейной функции распределения признаков. ROC-кривая также используется для оценки качества классификатора, который может быть использован для решения задач, в которых необходимо определи

## Вопрос №2: Чем отличается L2 регуляризация от L1 регуляризации?

In [ ]:
answer = rag.answer_with_rag('Чем отличается L2 регуляризация от L1 регуляризации?')

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(


In [ ]:
answer[1]

[{'score': 0.5741340335616256,
  'text': 'В параграфе «Регуляризация в онлайн-обучении» мы снова поговорим о регуляризации, но на этот раз речь пойдёт о регуляризаторах, которые накладывают на решение определённые органичения, например, разреженность. Вы сможете с новой стороны взглянуть на разреживающие свойства-регуляризаторов. Кроме того, мы получим не достижимые с помощью обычных SGD/AdaGrad результаты для разреженныхирегуляризаторов.',
  'article': 'Введение в онлайн-обучение',
  'segment_num': 9},
 {'score': 0.5729905223522018,
  'text': 'Регуляризаторвыбирается так, чтобы выражениебыло 1-сильно выпукло по отношению к некоторой норме(возможно, своей на каждом шаге).\nТогда\nгде— норма, двойственная к норме.\nПусть\nОбновление параметров происходит по правилу\nВыполнены все условия Setting 1;\nВсе регуляризаторылежат в семействе FTRL-Proximal, причёмдля всех;\nвыбирается так, чтобы выражениебыло 1-сильно выпукло по отношению к некоторой норме(возможно, своей на каждом шаге).\nТогд

In [ ]:
answer[0]

'L2 регуляризация и L1 регуляризация являются двух типов регуляризации в машинном обучении.\n\n1. L2 регуляризация:\n\nL2 регуляризация используется для регулирования векторов весов в модели, чтобы уменьшить размерность векторов. В L2 регуляризации вектор весов имеет максимальную длину, которая равна 1/2 * L2-norm( вектор весов).\n\n2. L1 регуляризация:\n\nL1 регуляризация используется для регулирования векторов весов, чтобы уменьшить их значение. В L1 регуляризации вектор весов имеет максимальную длину, которая равна 1/L1-norm( вектор весов).\n\nОбъяснение:\n\nВ L2 регуляризации вектор весов имеет максимальную длину, которая равна 1/2 * L2-norm( вектор весов). Это означает, что если вектор весов имеет длину 1, то его максимальная длина равна 1/2 * L2-norm( вектор весов). В L1 регуляризации вектор весов имеет максимальную длину, которая равна 1/L1-norm( вектор весов). Это означает, что если вектор весов имеет длину 1, то его максимальная длина равна 1/L1-norm( вектор весов).\n\nВ резул

In [ ]:
rag.answer_without_rag('Чем отличается L2 регуляризация от L1 регуляризации?')

'L2 регуляризация (L2 Regularization) и L1 регуляризация (L1 Regularization) отличаются в том, что L2 регуляризация использует векторную функцию, а L1 регуляризация использует векторную функцию с весом, который равен 1.0.\n\nL2 регуляризация используется для уменьшения разницы между весами и весами, которые были обучены на тестовых данных. Это позволяет обученным весам уменьшить разницу между весами, которые были обучены на тестовых данных и весами, которые были обучены на обучающей выборке.\n\nL1 регуляризация используется для уменьшения разницы между весами, которые были обучены на тестовых данных и весами, которые были обучены на обучающей выборке. Это позволяет обученным весам уменьшить разницу между весами, которые были обучены на тестовых данных и весами, которые были обучены на обучающей выборке.\n\nОбщие сведения о L2 регуляризации:\n- В L2 регуляризации векторная функция используется для уменьшения разницы между весами, которые были обучены на тестовых данных и весами, которые

## Вопрос №3: Что такое функция потерь?

In [ ]:
answer = rag.answer_with_rag('Что такое функция потерь?')

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(


In [ ]:
answer[1]

[{'score': 0.6863597437137183,
  'text': 'Функция потерь возникает в тот момент, когда мы сводим задачу построения модели к задаче оптимизации. Обычно требуется, чтобы она обладала хорошими свойствами (например, дифференцируемостью).\nФункция потерь возникает в тот момент, когда мы сводим задачу построения модели к задаче оптимизации. Обычно требуется, чтобы она обладала хорошими свойствами (например, дифференцируемостью).\nМетрика — внешний, объективный критерий качества, обычно зависящий не от параметров модели, а только от предсказанных меток.',
  'article': 'Метрики классификации и регрессии',
  'segment_num': 11},
 {'score': 0.6532116755668944,
  'text': 'В случае квадратичной функции потерь интуиция вполне подкрепляется математикой. Изменится ли что-либо в наших действиях, если мы поменяем квадратичную функцию потерь на любую другую? С одной стороны, мы, как и прежде, можем двигаться в направлении уменьшения разности предсказания и истинного значения: любая функция потерь поощряе

In [ ]:
answer[0]

'Функция потерь (loss function) в математической оптимизации является функцией, которая определяет качество решения задачи. В случае регрессии функция потерь может быть определена как функция от отклонения предсказанных значений от значений исходных данных. В случае классификации функция потерь может быть определена как функция от отклонения предсказанных классов от значений исходных классов.\n\nФункция потерь используется в качестве метрики качества в задачах регрессии и классификации. В регрессии функция потерь определяет качество решения задачи, а в классификации функция потерь определяет качество решения задачи.\n\nВ математической оптимизации функция потерь используется для определения оптимального решения задачи. В случае регрессии функция потерь используется для определения оптимального значения параметра, а в классификации функция потерь используется для определения оптимального класса.\n\nФункция потерь также используется в других областях, таких как оптимизация, а также в дру

In [ ]:
rag.answer_without_rag('Что такое функция потерь?')

'Функция потерь (loss function) - это функция, которая определяет качество или качество, которое требуется достичь в процессе обучения модели. В математическом смысле, функция потерь представляет собой функцию, которая определяет качество или качество, которое требуется достичь в процессе обучения модели. В языке программирования, функция потерь используется для определения качества модели, которая должна быть хорошо обучена.\n\nФункция потерь может быть различной в зависимости от задачи, которая используется для обучения модели. Например, в регрессии, функция потерь может быть функцией суммы квадратов ошибок (MSE), функцией суммы квадратов ошибок (MAE), функцией суммы квадратов ошибок (RMSE), функцией суммы квадратов ошибок (MSE), функцией суммы квадратов ошибок (MAPE), функцией суммы квадратов ошибок (RMSE), функцией суммы квадратов ошибок (MAPE), функцией суммы квадратов ошибок (RMSE), функцией суммы квадратов ошибок (MAPE), функцией суммы квадратов ошибок (RMSE), функцией суммы ква

## Вопрос №4: Чем отличается бустинг от бэггинга?

In [53]:
answer = rag.answer_with_rag('В чем разница между бэггингом и бустингом?')

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(


In [54]:
answer[1]

[{'score': 0.5538680788056545,
  'text': 'Рассмотрим две моды с силамии, такие что. Насколько вторая (более слабая) мода выучится к моменту, когда первая уже выучится на долю? Из уравнения выше имеем:\nПодставляяи, получаем:\nПоскольку, это выражение стремится к минус бесконечности при, из чего следует, чтостремится к нулю.',
  'article': 'Implicit bias',
  'segment_num': 9},
 {'score': 0.5021280466152587,
  'text': 'Состояние игрока — это сложное понятие, но, вероятно, мы можем выразить его, зная пульс, давление и другие физические показатели. В свою очередь, ситуацию на поле можно описать, как функцию от позиций и движений других игроков, судьи и зрителей — но всего не перечислишь, поэтому нам снова придётся привлекать случайность. Таким образом, мы получаем то, что называетсяграфической моделью:',
  'article': 'Вероятностный подход в ML',
  'segment_num': 16},
 {'score': 0.4990544110693901,
  'text': 'Heavy-ball Momentum: используется для ускорения процесса оптимизации. В выпуклых з

In [55]:
answer[0]

'Бэггинг и бустингом (Bagging и Boosting) являются двух основных методами в стохастической оптимизации.\n\n1. Бэггинг:\nБэггинг (Bagging) - это метод, который использует несколько экземпляров (всего N) кластеров для обучения. В каждом кластере обучается только один экземпляр, а затем эти экземпляры используются для обучения остальных кластеров. Этот метод используется для улучшения скорости сходимости и уменьшения размерности выборки.\n\n2. Бустингом:\nБустингом (Boosting) - это метод, который использует несколько экземпляров (всего N) кластеров для обучения. В каждом кластере обучается только один экземпляр, а затем эти экземпляры используются для обучения остальных кластеров. Этот метод используется для улучшения скорости сходимости и уменьшения размерности выборки.\n\nВажное отличие между этими методами заключается в том, что в бустинге используется несколько экземпляров кластеров, а не один. Это позволяет уменьшить размерность выборки, что в свою очередь позволяет улучшить скорость

In [56]:
rag.answer_without_rag('В чем разница между бэггингом и бустингом?')

'Бэггингом и бустингом относятся к разным методам тестирования программного обеспечения.\n\n1. Бэггинг: это метод тестирования, в котором программный код тестируется в режиме реального времени, то есть он выполняется в среде разработки. Этот метод позволяет определить, какие части программы работают нормально и какие нет.\n\n2. Бустинг: это метод тестирования, в котором программный код тестируется в режиме отладки, то есть он выполняется в среде отладки. Этот метод позволяет определить, какие части программы работают нормально и какие нет.\n\nОбщая разница между этими методами заключается в том, что бэггинг позволяет определить, какие части программы работают нормально и какие нет, то есть он позволяет тестировать программный код в режиме реального времени. Бустинг, в свою очередь, позволяет определить, какие части программы работают нормально и какие нет, то есть он позволяет тестировать программный код в режиме отладки.'

## Вопрос №5: Что такое LogLoss и зачем используется?

In [61]:
answer = rag.answer_with_rag('Что такое логарифмическая функция потерь и зачем используется?')

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(


In [62]:
answer[1]

[{'score': 0.6244355807364056,
  'text': 'Модель обучается на стандартную функцию потерь (например, LogLoss).\nИспользуя вещественные предсказания на валидационной выборке, перебирая разные пороги от 0 до 1, получаем графики метрик в зависимости от порога.\nВыбираем нужное сочетание точности и полноты.',
  'article': 'Метрики классификации и регрессии',
  'segment_num': 73},
 {'score': 0.6106060670086974,
  'text': 'Регуляризационный член не зависит от выборки и добавляется отдельно:\nСоответственно, идеальный градиент регуляризованной функции потерь имеет вид\nГрадиент по батчу – это тоже оценка градиента идеальной функции потерь, только не на выборке, а на батчеразмера. Он будет выглядеть так:',
  'article': 'Линейные модели',
  'segment_num': 79},
 {'score': 0.6040932804460828,
  'text': 'Бинарную классификацию рекомендовать/не рекомендовать. Тогдаимеет смысл логита, и мы можем оптимизировать оптимизировать log loss или hinge loss.\nРанжирование объектов. Тогда– это ранжирующая функ

In [63]:
answer[0]

'Логарифмическая функция потерь (LogLoss) используется в моделировании и оптимизации алгоритмов для решения задач машинного обучения.\n\nОна представляет собой функцию, которая оценивает качество модели на выборке, используя значение функции потерь на базовой выборке. Логарифмическая функция потерь используется для оценки качества модели, учитывая различия в размере выборки и размерности векторов входных данных.\n\nВыборка, используемая для оценки качества модели, называется базовой выборкой. Логарифмическая функция потерь используется для оценки качества модели на этой выборке.\n\nОсновная причина использования логарифмической функции потерь в моделировании и оптимизации алгоритмов для решения задач машинного обучения заключается в том, что она позволяет определить, насколько хорошо модель работает на данной выборке, и если она работает хорошо, то она может быть использована для обучения модели на других выборках.\n\nОднако, в некоторых случаях, когда базовая выборка не является доста

In [64]:
rag.answer_without_rag('Что такое логарифмическая функция потерь и зачем используется?')

"Логарифмическая функция потерь (Laplace's function of loss)  является функцией, которая определяет потерю в случае, когда потерь неизвестна. В данном случае, потерь можно определить как функцию потерь, которая принимает в качестве аргумента значение потерь, а затем вычисляет потерю в случае, когда потерь неизвестна.\n\nВ примере, когда потерь неизвестна, функция потерь используется для определения потерь в случае, когда потерь неизвестна. В данном случае, функция потерь используется для определения потерь в случае, когда потерь неизвестна.\n\nПример:\n\nДаны следующие данные:\n\n- Потерь в случае, когда потерь неизвестна: 10\n- Потерь в случае, когда потерь известна: 15\n\n- Потерь, которая принимает значение 10 в случае, когда потерь неизвестна: 10\n- Потерь, которая принимает значение 15 в случае, когда потерь известна: 15\n\nВ результате, функция потерь определяет потерю в случае, когда потерь неизвестна."

## Вопрос №6: Как работают свёрточные нейронные сети??

In [84]:
answer = rag.answer_with_rag('Объясни, как работают свёрточные нейронные сети?')

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(


In [85]:
answer[1]

[{'score': 0.7812052110251886,
  'text': 'Основные определения\n========================================\nИскусственная нейронная сеть (далее — нейронная сеть)— это сложная дифференцируемая функция, задающая отображение из исходного признакового пространства в пространство ответов, все параметры которой могут настраиваться одновременно и взаимосвязанно (то есть сеть может обучаться end-to-end).\nВ частном (и наиболее частом) случае представляет собой последовательность дифференцируемых параметрических преобразований.',
  'article': 'Первое знакомство с полносвязными нейросетями',
  'segment_num': 0},
 {'score': 0.7505699426525361,
  'text': 'В качестве иллюстрации ниже приведены структуры агностических нейронных сетей WANN, представленных в работеWeight Agnostic Neural Networks, NeurIPS 2019.\nForward & backward propagation\n========================================\nИнформация может течь по графу в двух направлениях.',
  'article': 'Первое знакомство с полносвязными нейросетями',
  'se

In [86]:
answer[0]

'Свёрточные нейронные сети (SqueezeNet, ResNet, etc.) - это семейство нейронных сетей, основанных на свёрточных слоях. В свёрточных слоях используются слои с малым числом входных и выходных каналов, но большим числом входных и выходных узлов. Это позволяет снизить размерность изображения, что в свою очередь позволяет снизить размерность выходных данных.\n\nВ свёрточных нейронных сетах используются следующие схемы свёрточных слоев:\n\n1. Squeeze-and-Excitation (SE) - это схема свёрточных слоев, в которой каждый слой свёртывает входную матрицу, а затем вычисляет вектор смещения, который используется для вычисления векторов смещения для следующего слоя.\n\n2. Pointwise Convolution (PC) - это схема свёрточных слоев, в которой каждый слой производит конvolution с малым числом входных каналов и вычисляет вектор смещения.\n\n3. Squeeze-and-Attention (SA) - это схема свёрточных слоев, в которой каждый слой свёртывает входную матрицу, а затем вычисляет вектор смещения, который используется для 

In [88]:
rag.answer_without_rag('Объясни, как работают свёрточные нейронные сети?')

'Свёрточные нейронные сети (SqueezeNet) - это один из наиболее популярных свёрточных нейронных сетей, разработанных в 2016 году. В свёрточных нейронных сетах используется смесь свёрточных и несвёрточных нейронов, которые работают в смеси сверху и снизу.\n\nСвёрточные нейроны - это нейроны, которые имеют сверху и снизу входные и выходные каналы, но не имеют входных и выходных каналов внутри себя. Это позволяет снизить размерность входных и выходных каналов, что в свою очередь позволяет снизить размерность выходных данных.\n\nВ свёрточных нейронных сетах используется смесь сверху и снизу свёрточных нейронов. В сверху свёрточные нейроны работают с входными и выходными каналами, которые разделены на две части. Входные каналы соединены свёрточными нейронами, которые обрабатывают данные внутри себя. Выходные каналы соединены свёрточными нейронами, которые обрабатывают данные внутри себя.\n\nВ свёрточных нейронных сетах используется смесь сверху и снизу свёрточных нейронов, что позволяет сниз

## Вопрос №7: Как работает механизм внимания?

In [94]:
answer = rag.answer_with_rag('Как работает механизм внимания?')

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(


In [95]:
answer[1]

[{'score': 0.7865614502891869,
  'text': 'Механизм внимания(attention) реализует эту интуицию путем предоставления декодеру информации обо всех токенах исходного предложения на каждом шаге генерации. Рассмотрим классическую модель внимания, предложенную Bahdanau et al. в 2014 году.',
  'article': 'Нейросети для работы с последовательностями',
  'segment_num': 72},
 {'score': 0.7378263160129224,
  'text': 'Механизм внутреннего внимания(self-attention) используется, чтобы посмотреть на другие слова во входной последовательности во время кодирования конкретного слова. Изначально этот механизм был представлен встатьеAttention is all you need как элемент архитектуры «трансформер» (Transformer).',
  'article': 'Нейросети для работы с последовательностями',
  'segment_num': 77},
 {'score': 0.712755813560159,
  'text': 'Существует много разных видов механизмов внимания, например:\nБазовый dot-product, рассмотренный ранее:\nМультипликативный:, где— обучаемая матрица весов.\nMLP:, где,— обучаемы

In [96]:
answer[0]

'В механизме внимания (Attention) используется предоставление декодеру информации обо всех токенах исходного предложения на каждом шаге генерации. Этот механизм реализуется путем предоставления декодеру информации обо всех токенах исходного предложения на каждом шаге генерации.\n\nДекодер получает вход из входной последовательности, которая состоит из двух частей:\n\n1. Входной последовательность (Input sequence) - это входная последовательность, которая состоит из двух частей:\n\n- Входной слова (Input word) - это каждый из слова в исходном предложении.\n- Входной вектор (Input vector) - это вектор с весами, которые используются для обучения механизма внимания.\n\n2. Декодер получает входную последовательность и вектор входных данных.\n\nДекодер использует предоставленный контекст для получения информации о связях между словами в двух языках, участвующих в переводе.\n\nВесь процесс генерации заключается в том, чтобы декодер получал входную последовательность и вектор входных данных, а

In [97]:
rag.answer_without_rag('Как работает механизм внимания?')

"The mechanism of attention is a complex process that involves the brain's frontal lobes, which are responsible for regulating and controlling attention. Attention is a critical cognitive function that allows us to focus our attention on specific tasks or objects, and it is essential for our daily lives.\n\nThe mechanism of attention works by detecting and processing information that is relevant to our current task or goal. When we encounter a stimulus that is relevant to our attention, our frontal lobes activate, and we focus our attention on that stimulus. This process is called selective attention.\n\nOnce we have focused our attention on a stimulus, our frontal lobes continue to monitor the stimulus and adjust our attention as needed. For example, if we are reading a book and come across a sentence that is relevant to our current task, our frontal lobes will activate and focus our attention on that sentence. If we are interrupted by a phone call or a distracting noise, our frontal 

## Вопрос №7: Объясни принцип работы случайного леса

In [98]:
answer = rag.answer_with_rag('Объясни принцип работы случайного леса')

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(


In [99]:
answer[1]

[{'score': 0.7303675852727778,
  'text': 'Чтобы получить предсказание ансамбля на тестовом объекте, усредняем отдельные ответы деревьев (для регрессии) или берём самый популярный класс (для классификации).\nProfit. Мы построилиRandom Forest (случайный лес)— комбинацию бэггинга и метода случайных подпространств над решающими деревьями.\nProfit. Мы построилиRandom Forest (случайный лес)— комбинацию бэггинга и метода случайных подпространств над решающими деревьями.',
  'article': 'Ансамбли в машинном обучении',
  'segment_num': 18},
 {'score': 0.7001641233160569,
  'text': 'Вторым практическим ограничением на количество деревьев может быть время работы ансамбля. Однако есть положительное свойство случайного леса: случайный лес можно строить и применять параллельно, что сокращает время работы, если у нас есть несколько процессоров. Но процессоров, скорее всего, всё же сильно меньше числа деревьев, а сами деревья обычно глубокие. Поэтому на большом числе деревьев Random Forest может работа

In [100]:
answer[0]

'Всем привет! В этом ответе расскажу, как работает случайный лес в машинном обучении.\n\nСлучайный лес - это алгоритм, который используется для построения регрессии или классификации. Он основан на идее, что в случае, когда в данных есть много признаков, то можно использовать только несколько из них для построения модели.\n\nДля построения случайного леса используется метод случайных подпространств над решающими деревьями. Этот метод позволяет построить случайный лес, который будет содержать только несколько деревьев, но будет иметь высокую точность и скорость работы.\n\nВ качестве решающего дерева случайный лес использует случайные подпространства, которые являются подпространствами решающего дерева. В качестве подпространства случайный лес использует случайные подпространства, которые являются подпространствами решающего дерева.\n\nВ качестве решающего дерева случайный лес использует случайные подпространства, которые являются подпространствами решающего дерева. В качестве подпростра

In [101]:
rag.answer_without_rag('Объясни принцип работы случайного леса')

'Суть принципа работы случайного леса заключается в том, что в нём происходит неопределенное сочетание различных элементов, которые взаимодействуют друг с другом в определённой последовательности. В случае случайного леса эти элементы могут быть разными по типу, размеру, цвету, расположению и\xa0т.\xa0д.\n\nПринцип работы случайного леса основан на том, что в нём происходит неопределенное сочетание различных элементов, которые взаимодействуют друг с другом в определённой последовательности. В случае случайного леса эти элементы могут быть разными по типу, размеру, цвету, расположению и\xa0т.\xa0д.\n\nОбъяснение принципа работы случайного леса можно найти в книге "Суперпозиция и случайный лес" (англ. "Superposition and Random Forest") автора Джона К. Смита. В ней автор рассматривает случайный лес как упрощённый вариант суперпозиции, которая также является методом классификации данных.\n\nВ суперпозиции элементы могут быть разными по типу, размеру, цвету, расположению и\xa0т.\xa0д. В слу

## Вопрос №8

In [108]:
question = 'Что такое градиентный бустинг?'
answer = rag.answer_with_rag(question)
print(f"\n\nВопрос: {question}\n\n")
print(f"Информация из векторной базы данных: {answer[1]}\n\n")
print(f"Ответ модели с RAG: {answer[0]}\n\n")
print(f"Ответ модели без RAG: {rag.answer_without_rag(question)}\n\n")

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(




Вопрос: Что такое градиентный бустинг?


Информация из векторной базы данных: [{'score': 0.580110389938626, 'text': 'где– это дисперсия стохградиента, а– константа сильной выпуклости, показывающая, насколько функция является «не плоской» в окрестности точки оптимума. Доказательство в том жепрепринте С. Стича.\nМораль в следующем: дисперсия стохастического градиента, вычисленного по батчу размераравна, где– это дисперсия одного градиента. То есть увеличение размера батча помогает и с теоретической точки зрения.', 'article': 'Оптимизация в ML', 'segment_num': 39}, {'score': 0.5784129723220435, 'text': 'Для объяснения метода градиентного бустинга полезно воспользоваться следующей аналогией. Бустинг можно представить как гольфиста, цель которого — загнать мяч в лунку с координатой. Положение мяча здесь – ответ композиции. Гольфист мог бы один раз ударить по мячу, не попасть в лунку и пойти домой, но настырность заставляет его продолжить.', 'article': 'Градиентный бустинг', 'segment_num':

## Вопрос №9

In [114]:
question = "Чем XGBoost отличается от LightGBM?"
answer = rag.answer_with_rag(question)
print(f"\n\nВопрос: {question}\n\n")
print(f"Информация из векторной базы данных: {answer[1]}\n\n")
print(f"Ответ модели с RAG: {answer[0]}\n\n")
print(f"Ответ модели без RAG: {rag.answer_without_rag(question)}\n\n")

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(




Вопрос: Чем XGBoost отличается от LightGBM?


Информация из векторной базы данных: [{'score': 0.6915209880402512, 'text': 'Хороших реализаций GBDT есть, как минимум, три:LightGBM,XGBoostиCatBoost. Исторически они отличались довольно сильно, но за последние годы успели скопировать друг у друга все хорошие идеи.\nОдно из основных отличий LightGBM, XGBoost и CatBoost — форма решающих деревьев.', 'article': 'Градиентный бустинг', 'segment_num': 47}, {'score': 0.6061749739187492, 'text': 'С одной стороны, это позволяет быстро подогнаться под обучающие данные. С другой — бесконтрольный рост дерева в глубину неизбежно ведет к переобучению, поэтому LightGBM позволяет помимо количества вершин ограничивать и максимальную глубину. Впрочем, это ограничение обычно все равно выше, чем для XGBoost и CatBoost.', 'article': 'Градиентный бустинг', 'segment_num': 49}, {'score': 0.49852047327580534, 'text': 'Это один из важнейших компонент всвёрточныхнейронных сетях. Веса свёртки, упорядоченные в тензор

## Вопрос №10

In [115]:
question = "Как работает метод опорных векторов?"
answer = rag.answer_with_rag(question)
print(f"\n\nВопрос: {question}\n\n")
print(f"Информация из векторной базы данных: {answer[1]}\n\n")
print(f"Ответ модели с RAG: {answer[0]}\n\n")
print(f"Ответ модели без RAG: {rag.answer_without_rag(question)}\n\n")

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(




Вопрос: Как работает метод опорных векторов?


Информация из векторной базы данных: [{'score': 0.7394976401360729, 'text': 'Итоговое положение плоскости задаётся всего несколькими обучающими примерами. Это ближайшие к плоскости правильно классифицированные объекты, которые называютопорными векторамиилиsupport vectors. Весь метод, соответственно, зовётся методомопорных векторов, илиsupport vector machine, или сокращённоSVM. Начиная с шестидесятых годов это был сильнейший из известных методов машинного обучения. В девяностые его сменили методы, основанные на деревьях решений, которые, в свою очередь, недавно передали «пальму первенства» нейросетям.', 'article': 'Линейные модели', 'segment_num': 109}, {'score': 0.7019017736401407, 'text': 'Представим себе самую простую модель, основанную на данном принципе — что-то вроде ансамбля линейных. Каждую из сдвинутых картинок вытянем в вектор и скалярно умножим на вектор весов (для простоты один и тот же для всех сдвигов) — получим линейный опе

## Вопрос №11

In [116]:
question = "Что такое TF-IDF и как он применяется в NLP? "
answer = rag.answer_with_rag(question)
print(f"\n\nВопрос: {question}\n\n")
print(f"Информация из векторной базы данных: {answer[1]}\n\n")
print(f"Ответ модели с RAG: {answer[0]}\n\n")
print(f"Ответ модели без RAG: {rag.answer_without_rag(question)}\n\n")

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(




Вопрос: Что такое TF-IDF и как он применяется в NLP? 


Информация из векторной базы данных: [{'score': 0.42880618730996956, 'text': 'Значение классификаторов\nМногие модели могут подстроиться под эти факторы и отдельно обучиться под различные значения запросных признаков. Например, так модель может по-разному реагировать на запросы из разных стран.\nЧтобы сопоставить запрос и документ, можно использовать TF-IDF слов запроса. Например, можно просуммировать его по всем словам из запроса и получить фактор для ранжирования. Подробно о том, как считать TF-IDF, можно прочитать в главе про NLP.', 'article': 'Задача ранжирования', 'segment_num': 61}, {'score': 0.36777754896963255, 'text': 'SFT (supervised finetuning) — обучение модели следовать инструкциям. Этот пункт мы подробно обсудили в предыдущей части параграфа (T0, FLAN, CoT). На этом этапе важно составить грамотный инструкционный датасет, где инструкция содержит произвольные запросы к модели, а ответ на неё — подробный текст, которы

## Вопрос №12

In [117]:
question = "Что такое кросс-валидация?"
answer = rag.answer_with_rag(question)
print(f"\n\nВопрос: {question}\n\n")
print(f"Информация из векторной базы данных: {answer[1]}\n\n")
print(f"Ответ модели с RAG: {answer[0]}\n\n")
print(f"Ответ модели без RAG: {rag.answer_without_rag(question)}\n\n")

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(




Вопрос: Что такое кросс-валидация?


Информация из векторной базы данных: [{'score': 0.7646141532130247, 'text': 'Кросс-валидация — это процедура для оценки качества работы модели, которая широко применяется в машинном обучении. Она помогает сравнить между собой различные модели и выбрать наилучшую для конкретной задачи.\nВ этом разделе мы рассмотрим наиболее распространённые методы кросс-валидации, а также обсудим возможные проблемы, которые могут возникнуть в процессе их применения.\nHold-out\n========================================\nМетодhold-outпредставляет из себя простое разделение на train и test:', 'article': 'Кросс-валидация', 'segment_num': 0}, {'score': 0.6591472369690808, 'text': 'В sklearn реализована такая схема кросс-валидации:', 'article': 'Кросс-валидация', 'segment_num': 42}, {'score': 0.45562286854676354, 'text': 'Финальный скор модели получается либо усреднениемполучившихся тестовых результатов, либо измеряется на отложенном тестовом множестве, не участвовавшем в

## Вопрос №13

In [118]:
question = "Какие есть способы оптимизации гиперпараметров?"
answer = rag.answer_with_rag(question)
print(f"\n\nВопрос: {question}\n\n")
print(f"Информация из векторной базы данных: {answer[1]}\n\n")
print(f"Ответ модели с RAG: {answer[0]}\n\n")
print(f"Ответ модели без RAG: {rag.answer_without_rag(question)}\n\n")

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(




Вопрос: Какие есть способы оптимизации гиперпараметров?


Информация из векторной базы данных: [{'score': 0.7513921441506308, 'text': 'Подбор гиперпараметров тоже можно сформулировать в виде задачи, которая может решаться с помощью байесовской оптимизации. Пусть, например, наша функция — значение валидационных метрик в зависимости от текущего сочетания гиперпараметров. Её вычисление затратно по времени (нужно натренировать и провалидировать модель), и мы не можем вычислить градиенты этой функции по её переменным (нашим гиперпараметрам).\nБайесовская оптимизация имеет две основные компоненты:', 'article': 'Подбор гиперпараметров', 'segment_num': 26}, {'score': 0.7165964741635888, 'text': 'Самый естественный способ организовать перебор наборов гиперпараметров — сделать перебор по сетке (Grid Search):\nдля каждого гиперпараметра фиксируется несколько значений;\nперебираются все комбинации значений различных гиперпараметров, на каждой из этих комбинаций модель обучается и тестируется;\nв

## Вопрос №14

In [125]:
question = "Как работает алгоритм k-ближайших соседей?"
answer = rag.answer_with_rag(question)
print(f"\n\nВопрос: {question}\n\n")
print(f"Информация из векторной базы данных: {answer[1]}\n\n")
print(f"Ответ модели с RAG: {answer[0]}\n\n")
print(f"Ответ модели без RAG: {rag.answer_without_rag(question)}\n\n")

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(




Вопрос: Как работает алгоритм k-ближайших соседей?


Информация из векторной базы данных: [{'score': 0.6919798215059092, 'text': 'В первой части параграфа мы расскажем об одном из самых известных метрических алгоритмов — методеk-ближайших соседей(k-nearest neighbors, KNN). Этот подход в основном чисто инженерный из-за отсутствия фазы обучения — в настоящее время уже почти нигде не применяется. Однако многие техники, на которых основан алгоритм, используются и в других методах.', 'article': 'Метрические методы', 'segment_num': 3}, {'score': 0.6919722878195854, 'text': 'Поиск ближайших соседей\n========================================\nДля того чтобы применять метод ближайших соседей, нужно уметь как-то находить этих самых соседей. С первого взгляда может показаться, что никакой проблемы нет: действительно, можно ведь просто перебрать все объекты из обучающей выборки, посчитать для каждого из них расстояние до тестового объекта и затем найти минимум.', 'article': 'Метрические методы', 

## Вопрос №15

In [126]:
question = "Что такое матрица ошибок?"
answer = rag.answer_with_rag(question)
print(f"\n\nВопрос: {question}\n\n")
print(f"Информация из векторной базы данных: {answer[1]}\n\n")
print(f"Ответ модели с RAG: {answer[0]}\n\n")
print(f"Ответ модели без RAG: {rag.answer_without_rag(question)}\n\n")

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(




Вопрос: Что такое матрица ошибок?


Информация из векторной базы данных: [{'score': 0.6718861867535283, 'text': 'Вывод: матрица, ортонормированная по столбцам, отвечает датасету, в котором признаки не коррелированы и имеют единичную дисперсию\nСингулярное разложение\n========================================', 'article': 'Матричная факторизация', 'segment_num': 18}, {'score': 0.6572008316798352, 'text': 'Весь набор ошибок на отложенной выборке может служить аналогом матрицы ошибок из задачи классификации. А именно, когда мы рассматриваем две разные модели, то, глядя на то, как и на каких объектах они ошиблись, мы можем прийти к выводу, что для решения бизнес-задачи нам выгоднее взять ту или иную модель. И, аналогично со случаем бинарной классификации, мы можем начать строить агрегаты от вектора ошибок, получая тем самым разные метрики.\nMSE, RMSE,\n========================================', 'article': 'Метрики классификации и регрессии', 'segment_num': 81}, {'score': 0.644883180848368

## Вопрос №16

In [128]:
question = "Что такое графовая нейронная сеть?"
answer = rag.answer_with_rag(question)
print(f"\n\nВопрос: {question}\n\n")
print(f"Информация из векторной базы данных: {answer[1]}\n\n")
print(f"Ответ модели с RAG: {answer[0]}\n\n")
print(f"Ответ модели без RAG: {rag.answer_without_rag(question)}\n\n")

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(




Вопрос: Что такое графовая нейронная сеть?


Информация из векторной базы данных: [{'score': 0.8329692450605349, 'text': 'Как говорилось ранее, графовые нейронные сети являются обобщением сверточных. Если представить пиксели изображения вершинами графа, соединить соседние по свертке пиксели ребрами и предоставить относительную позицию пикселей в информации о ребре, то графовая свертка на таком графе будет работать так же, как и свертка над изображением.', 'article': 'Графовые нейронные сети', 'segment_num': 17}, {'score': 0.8019747025591457, 'text': 'Однако, сама суть работы у графовых и сверточных сетей совпадает. В графовой нейронной сети по очереди применяются слои, которые собирают информацию с соседей  и обновляют информацию в вершине. То же самое делают и обычные свертки. Поэтому такие слои и называютсяграфовыми свертками. Графовая свертка принимает на вход граф со скрытыми состояниями у вершин и ребер и выдает тот же граф, но уже с обновленными более информативными скрытыми со

## Вопрос №17

In [130]:
question = "Что такое иерархическая кластеризация?"
answer = rag.answer_with_rag(question)
print(f"\n\nВопрос: {question}\n\n")
print(f"Информация из векторной базы данных: {answer[1]}\n\n")
print(f"Ответ модели с RAG: {answer[0]}\n\n")
print(f"Ответ модели без RAG: {rag.answer_without_rag(question)}\n\n")

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(




Вопрос: Что такое иерархическая кластеризация?


Информация из векторной базы данных: [{'score': 0.863767445907889, 'text': 'Иерархическая агломеративная кластеризация\n========================================\nДругой классический метод кластеризации — этоиерархическая кластеризация. Иногда дополнительно уточняют:иерархическая агломеративная кластеризация. Название указывает сразу на два обстоятельства.', 'article': 'Кластеризация', 'segment_num': 35}, {'score': 0.8114915762617518, 'text': 'Во-вторых, кластеризация бывает, по аналогии с оргструктурой в организациях, плоской (когда все кластеры равноправны и находятся на одном уровне кластеризации) и иерархической (когда кластеры бывают вложены друг в друга и образуют древовидную структуру).', 'article': 'Кластеризация', 'segment_num': 37}, {'score': 0.7985560043884936, 'text': 'В случае иерархической агломеративной кластеризации мы действительно будем начинать с кластеров из одного объекта, постепенно объединяя их, а уже последовател

## Вопрос №18

In [131]:
question = "Что такое коэффициент силуэта?"
answer = rag.answer_with_rag(question)
print(f"\n\nВопрос: {question}\n\n")
print(f"Информация из векторной базы данных: {answer[1]}\n\n")
print(f"Ответ модели с RAG: {answer[0]}\n\n")
print(f"Ответ модели без RAG: {rag.answer_without_rag(question)}\n\n")

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(




Вопрос: Что такое коэффициент силуэта?


Информация из векторной базы данных: [{'score': 0.541811234130438, 'text': 'Чтобы ввести коэффициент силуэта, нам потребуются две вспомогательные величины. Первая,, — это среднее расстояние междуи объектами того же кластера. Вторая,, — это среднее расстояние междуи объектами следующего ближайшего кластера. Коэффициент силуэта вводится следующим образом:', 'article': 'Кластеризация', 'segment_num': 63}, {'score': 0.5255612946413701, 'text': 'где– это дисперсия стохградиента, а– константа сильной выпуклости, показывающая, насколько функция является «не плоской» в окрестности точки оптимума. Доказательство в том жепрепринте С. Стича.\nМораль в следующем: дисперсия стохастического градиента, вычисленного по батчу размераравна, где– это дисперсия одного градиента. То есть увеличение размера батча помогает и с теоретической точки зрения.', 'article': 'Оптимизация в ML', 'segment_num': 39}, {'score': 0.5251804141042365, 'text': 'Слабости метода Ньюто

## Вопрос №19

In [138]:
question = "Что такое энтропия?"
answer = rag.answer_with_rag(question)
print(f"\n\nВопрос: {question}\n\n")
print(f"Информация из векторной базы данных: {answer[1]}\n\n")
print(f"Ответ модели с RAG: {answer[0]}\n\n")
print(f"Ответ модели без RAG: {rag.answer_without_rag(question)}\n\n")

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(




Вопрос: Что такое энтропия?


Информация из векторной базы данных: [{'score': 0.7349346983672955, 'text': 'Кстати, Шеннон изначально собирался назвать информационную энтропию или «информацией», или «неопределённостью», но в итоге выбрал название «энтропия», потому что концепция со схожим смыслом в статистической механике уже была названа энтропией. Употребление термина из другой научной области выглядело убедительным преимуществом при ведении научных споров.\nИнформативность в задаче классификации: критерий Джини\n========================================', 'article': 'Решающие деревья', 'segment_num': 46}, {'score': 0.7303112958867513, 'text': 'Вот мы и вывели формулу энтропии!\nПример 2. Энтропия нормального распределенияравна, и чем меньше дисперсия, тем меньше энтропия, что и логично: ведь когда дисперсия мала, значения сосредоточены возле матожидания, и они становятся менее «разнообразными».\nЭнтропия тесно связана с другим важным понятием из теории информации —дивергенцией Кульб

## Вопрос №20

In [135]:
question = "Что такое MSE?"
answer = rag.answer_with_rag(question)
print(f"\n\nВопрос: {question}\n\n")
print(f"Информация из векторной базы данных: {answer[1]}\n\n")
print(f"Ответ модели с RAG: {answer[0]}\n\n")
print(f"Ответ модели без RAG: {rag.answer_without_rag(question)}\n\n")

<ipython-input-31-e65552691808>:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(




Вопрос: Что такое MSE?


Информация из векторной базы данных: [{'score': 0.6897643678364391, 'text': 'MSE — одна из самых популярных метрик в задаче регрессии. Она уже знакома вам, так как применяется в качестве функции потерь (или входит в ее состав) во многих ранее рассмотренных методах.\nИногда для того, чтобы показатель эффективности MSE имел размерность исходных данных, из него извлекают квадратный корень и получают показатель эффективности RMSE.', 'article': 'Метрики классификации и регрессии', 'segment_num': 82}, {'score': 0.6149995752401051, 'text': 'При фиксированной дисперсии и при предсказании шума с помощью взвешенной MSE. Или просто MSE.являетсясамым популярнымвариантом, который на практике дает лучшие результаты.\nВыбор расписания\n========================================\nРасписание является гиперпараметром, основными требованиями на который являются невозрастаниеи чтобы прямой процесс сходился кв пределе по. Второе может гарантироваться тем, что. Вспомним,\nОднако на 